In [3]:
import pandas as pd
import sqlite3
from pathlib import Path

# Path to data folder
data_dir = Path("data_maps")

# Connect (or create) SQLite database
conn = sqlite3.connect("ads_data.db")
cursor = conn.cursor()

print("Database connected successfully.")


Database connected successfully.


In [4]:
# Load campaign ID → name
campaign_file = data_dir / "campaign_id_to_name_map.csv"
campaigns = pd.read_csv(campaign_file)

# Save to SQLite
campaigns.to_sql('Campaign', conn, if_exists='replace', index=False)

print("Campaign table created with", len(campaigns), "rows.")


Campaign table created with 14774 rows.


In [6]:
# Load ad group ID → name
ad_group_file = data_dir / "ad_group_id_to_name_map.csv"
ad_groups = pd.read_csv(ad_group_file)

# Load campaign → ad group mapping (IDs)
camp_ad_group_file = data_dir / "campaign_id_to_ad_group_id_map.csv"
camp_ad_groups = pd.read_csv(camp_ad_group_file)

# Merge to get campaign_id in ad_groups
ad_groups_full = ad_groups.merge(camp_ad_groups, on='Ad Group ID', how='left')

# Save to SQLite
ad_groups_full.to_sql('AdGroup', conn, if_exists='replace', index=False)

print("AdGroup table created with", len(ad_groups_full), "rows.")


AdGroup table created with 20037 rows.


In [7]:
# Load keyword ID → keyword text
keyword_file = data_dir / "keyword_id_to_name_map.csv"
keywords = pd.read_csv(keyword_file)

# Save to SQLite
keywords.to_sql('Keyword', conn, if_exists='replace', index=False)

print("Keyword table created with", len(keywords), "rows.")


Keyword table created with 638323 rows.


In [8]:
# Load campaign → negative keyword mapping
neg_kw_file = data_dir / "campaign_to_negative_keyword_mapping.csv"
neg_keywords = pd.read_csv(neg_kw_file)

# Save to SQLite
neg_keywords.to_sql('NegativeKeyword', conn, if_exists='replace', index=False)

print("NegativeKeyword table created with", len(neg_keywords), "rows.")


NegativeKeyword table created with 21710 rows.


In [13]:
# Load SKU → campaign mapping
sku_campaign_file = data_dir / "sku_to_campaign_name_map.csv"
sku_campaign = pd.read_csv(sku_campaign_file)

# Load ASIN → campaign mapping
asin_campaign_file = data_dir / "asin_informational_only_to_name_map.csv"
asin_campaign = pd.read_csv(asin_campaign_file)

# Clean column names for convenience
sku_campaign.columns = ['sku', 'campaign_name']
asin_campaign.columns = ['asin', 'campaign_name']

# Save SKU table
sku_campaign[['sku']].drop_duplicates().to_sql('SKU', conn, if_exists='replace', index=False)

# Save CampaignSKU mapping
sku_campaign.to_sql('CampaignSKU', conn, if_exists='replace', index=False)

# Save ASIN table
asin_campaign[['asin']].drop_duplicates().to_sql('ASIN', conn, if_exists='replace', index=False)

# Save CampaignASIN mapping
asin_campaign.to_sql('CampaignASIN', conn, if_exists='replace', index=False)

print("SKU, ASIN, CampaignSKU, and CampaignASIN tables created.")


SKU, ASIN, CampaignSKU, and CampaignASIN tables created.


In [14]:
import pandas as pd
import sqlite3

# Connect to database
conn = sqlite3.connect("ads_data.db")

# Load campaign performance CSV
performance_file = "data/Sponsored_Products_Campaign_L30.csv"  # Replace with your actual file path
df = pd.read_csv(performance_file)

# Optional: clean column names
df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

# Create Campaign table
campaign_cols = ['campaign_name', 'portfolio_name', 'program_type', 'retailer',
                 'country', 'status', 'currency', 'budget_amount', 'Average Time in Budget','targeting_type', 'bidding_strategy']

campaign_table = df[campaign_cols].drop_duplicates(subset=['campaign_name'])
campaign_table.to_sql('Campaign', conn, if_exists='replace', index=False)

print("Campaign table populated with", len(campaign_table), "unique campaigns.")


Campaign table populated with 2232 unique campaigns.


In [15]:
# Select relevant performance metrics
perf_cols = ['campaign_name', 'start_date', 'end_date', 'impressions', 'last_year_impressions',
             'clicks', 'last_year_clicks', 'click-thru_rate_(ctr)', 'spend', 'last_year_spend',
             'cost_per_click_(cpc)', 'last_year_cost_per_click_(cpc)',
             '7_day_total_orders_(#)', 'total_advertising_cost_of_sales_(acos)', 
             'total_return_on_advertising_spend_(roas)', '7_day_total_sales']

perf_table = df[perf_cols]

# Save to SQLite
perf_table.to_sql('CampaignPerformance', conn, if_exists='replace', index=False)

print("CampaignPerformance table created with", len(perf_table), "rows.")


CampaignPerformance table created with 2232 rows.


In [17]:
search_terms_file = "data/Sponsored_Products_Search_Term_Detailed_L30.xlsx"  # Replace with your file path
df = pd.read_excel(search_terms_file)

# Clean column names for convenience
df.columns = [col.strip().replace(" ", "_").replace("(", "").replace(")", "").lower() for col in df.columns]

print("Columns loaded:", df.columns.tolist())
print("Number of rows:", len(df))

d:\work\n8n-work-flow\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Columns loaded: ['date', 'portfolio_name', 'currency', 'campaign_name', 'ad_group_name', 'retailer', 'country', 'targeting', 'match_type', 'customer_search_term', 'impressions', 'clicks', 'click-thru_rate_ctr', 'cost_per_click_cpc', 'spend', '7_day_total_sales', 'total_advertising_cost_of_sales_acos', 'total_return_on_advertising_spend_roas', '7_day_total_orders_#', '7_day_total_units_#', '7_day_conversion_rate', '7_day_advertised_sku_units_#', '7_day_other_sku_units_#', '7_day_advertised_sku_sales', '7_day_other_sku_sales']
Number of rows: 621867


In [18]:
# Select relevant columns for performance table
keyword_perf_cols = [
    'date', 'portfolio_name', 'currency', 'campaign_name', 'ad_group_name', 
    'retailer', 'country', 'targeting', 'match_type', 'customer_search_term',
    'impressions', 'clicks', 'click-thru_rate_ctr', 'cost_per_click_cpc', 
    'spend', '7_day_total_sales', 'total_advertising_cost_of_sales_acos', 
    'total_return_on_advertising_spend_roas', '7_day_total_orders_#', 
    '7_day_total_units_#', '7_day_conversion_rate', 
    '7_day_advertised_sku_units_#', '7_day_other_sku_units_#',
    '7_day_advertised_sku_sales', '7_day_other_sku_sales'
]

# Keep only these columns
keyword_perf_table = df[keyword_perf_cols]

# Convert numeric columns to proper types
numeric_cols = [
    'impressions', 'clicks', 'click-thru_rate_ctr', 'cost_per_click_cpc', 
    'spend', '7_day_total_sales', 'total_advertising_cost_of_sales_acos', 
    'total_return_on_advertising_spend_roas', '7_day_total_orders_#', 
    '7_day_total_units_#', '7_day_conversion_rate', 
    '7_day_advertised_sku_units_#', '7_day_other_sku_units_#',
    '7_day_advertised_sku_sales', '7_day_other_sku_sales'
]

for col in numeric_cols:
    keyword_perf_table[col] = pd.to_numeric(keyword_perf_table[col], errors='coerce')

# Save to SQLite
keyword_perf_table.to_sql('KeywordPerformance', conn, if_exists='replace', index=False)

print("KeywordPerformance table created with", len(keyword_perf_table), "rows.")


KeywordPerformance table created with 621867 rows.


In [19]:
def get_tables_and_columns(db_name):
    """
    Connects to a SQLite database and returns a dictionary where
    keys are table names and values are lists of column names.
    """
    tables_columns = {}
    
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Get all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]
            # Get column info for each table
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns_info = cursor.fetchall()
            column_names = [col[1] for col in columns_info]  # second field is column name
            tables_columns[table_name] = column_names

        conn.close()
        return tables_columns

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

# Example usage
db_path = "ads_data.db"
result = get_tables_and_columns(db_path)
print(result)

{'AdGroup': ['Ad Group ID', 'Ad Group Name (Informational only)', 'Campaign ID'], 'Keyword': ['Keyword ID', 'Keyword Text'], 'NegativeKeyword': ['campaign_name', 'negative_keyword_text'], 'SKU': ['sku'], 'CampaignSKU': ['sku', 'campaign_name'], 'ASIN': ['asin'], 'CampaignASIN': ['asin', 'campaign_name'], 'Campaign': ['campaign_name', 'portfolio_name', 'program_type', 'retailer', 'country', 'status', 'currency', 'budget_amount', 'targeting_type', 'bidding_strategy'], 'CampaignPerformance': ['campaign_name', 'start_date', 'end_date', 'impressions', 'last_year_impressions', 'clicks', 'last_year_clicks', 'click-thru_rate_(ctr)', 'spend', 'last_year_spend', 'cost_per_click_(cpc)', 'last_year_cost_per_click_(cpc)', '7_day_total_orders_(#)', 'total_advertising_cost_of_sales_(acos)', 'total_return_on_advertising_spend_(roas)', '7_day_total_sales'], 'KeywordPerformance': ['date', 'portfolio_name', 'currency', 'campaign_name', 'ad_group_name', 'retailer', 'country', 'targeting', 'match_type', 'c

In [24]:
import json

def get_campaign_summary(db_path, campaign_name):
    """
    Returns:
    - Campaign performance
    - Campaign portfolio info
    - Keywords used in the campaign (from KeywordPerformance)
    - Keyword performance
    """
    summary = {}

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # --- Campaign performance ---
        cursor.execute("""
            SELECT * FROM CampaignPerformance
            WHERE campaign_name = ?
        """, (campaign_name,))
        perf_columns = [desc[0] for desc in cursor.description]
        campaign_perf = cursor.fetchone()
        summary['campaign_performance'] = dict(zip(perf_columns, campaign_perf)) if campaign_perf else None

        # --- Campaign info ---
        cursor.execute("""
            SELECT * FROM Campaign
            WHERE campaign_name = ?
        """, (campaign_name,))
        info_columns = [desc[0] for desc in cursor.description]
        campaign_info = cursor.fetchone()
        summary['campaign_info'] = dict(zip(info_columns, campaign_info)) if campaign_info else None

        # --- Keyword performance (also gives keywords used) ---
        cursor.execute("""
            SELECT * FROM KeywordPerformance
            WHERE campaign_name = ?
        """, (campaign_name,))
        kw_perf_columns = [desc[0] for desc in cursor.description]
        keyword_perf_rows = cursor.fetchall()
        keyword_perf_list = [dict(zip(kw_perf_columns, row)) for row in keyword_perf_rows]
        summary['keyword_performance'] = keyword_perf_list

        # --- Keywords used ---
        # Extract unique customer_search_terms from keyword performance
        keywords_used = list({row['customer_search_term'] for row in keyword_perf_list})
        summary['keywords'] = keywords_used

        conn.close()
        return summary

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

# -------------------------
# Example usage:
# db_path = "your_database.db"
# campaign_name = "My Campaign Name"
# summary = get_campaign_summary(db_path, campaign_name)
# print(summary)
summary = get_campaign_summary('ads_data.db',"UTELT4BT2754-600GMBLACK - Group C")
print(json.dumps(summary, indent=4, sort_keys=True))


{
    "campaign_info": {
        "bidding_strategy": "Dynamic bids - down only",
        "budget_amount": "$30.0",
        "campaign_name": "UTELT4BT2754-600GMBLACK - Group C",
        "country": "United States",
        "currency": "USD",
        "portfolio_name": "Towel",
        "program_type": "Sponsored Products",
        "retailer": "Amazon",
        "status": "ENABLED",
        "targeting_type": "Manual targeting"
    },
    "campaign_performance": {
        "7_day_total_orders_(#)": 76,
        "7_day_total_sales": "$2964.02",
        "campaign_name": "UTELT4BT2754-600GMBLACK - Group C",
        "click-thru_rate_(ctr)": "1.121%",
        "clicks": 294,
        "cost_per_click_(cpc)": "$1.5181973",
        "end_date": "Jan 04, 2026",
        "impressions": 26236,
        "last_year_clicks": null,
        "last_year_cost_per_click_(cpc)": "$0.0",
        "last_year_impressions": null,
        "last_year_spend": null,
        "spend": "$446.35",
        "start_date": "Dec 06, 2025

In [25]:
def get_campaign_summary_with_keyword_score(db_path, campaign_name):
    summary = {}

    try:
        conn = sqlite3.connect(db_path)

        # --- Campaign performance ---
        campaign_perf = pd.read_sql_query(
            "SELECT * FROM CampaignPerformance WHERE campaign_name = ?", conn, params=(campaign_name,)
        )
        summary['campaign_performance'] = campaign_perf.to_dict(orient='records')[0] if not campaign_perf.empty else None

        # --- Campaign info ---
        campaign_info = pd.read_sql_query(
            "SELECT * FROM Campaign WHERE campaign_name = ?", conn, params=(campaign_name,)
        )
        summary['campaign_info'] = campaign_info.to_dict(orient='records')[0] if not campaign_info.empty else None

        # --- Keyword performance ---
        kw_perf = pd.read_sql_query(
            "SELECT * FROM KeywordPerformance WHERE campaign_name = ?", conn, params=(campaign_name,)
        )

        if kw_perf.empty:
            summary['keyword_performance'] = []
            summary['keywords'] = []
            conn.close()
            return summary

        # --- Aggregate by keyword ---
        kw_agg = kw_perf.groupby('customer_search_term').agg(
            impressions=pd.NamedAgg(column='impressions', aggfunc='sum'),
            clicks=pd.NamedAgg(column='clicks', aggfunc='sum'),
            spend=pd.NamedAgg(column='spend', aggfunc='sum'),
            sales=pd.NamedAgg(column='7_day_total_sales', aggfunc='sum'),
            orders=pd.NamedAgg(column='7_day_total_orders_#', aggfunc='sum')
        ).reset_index()

        # --- Compute CTR, CPC, ROAS ---
        kw_agg['ctr'] = kw_agg['clicks'] / kw_agg['impressions'].replace(0, 1)
        kw_agg['cpc'] = kw_agg['spend'] / kw_agg['clicks'].replace(0, 1)
        kw_agg['roas'] = kw_agg['sales'] / kw_agg['spend'].replace(0, 1)

        # --- Compute score out of 10 ---
        def compute_score(row):
            score = 0
            # CTR weighting (0-4 points)
            if row['ctr'] > 0.1:      score += 4
            elif row['ctr'] > 0.05:   score += 3
            elif row['ctr'] > 0.02:   score += 2
            elif row['ctr'] > 0:      score += 1

            # ROAS weighting (0-3 points)
            if row['roas'] > 5:       score += 3
            elif row['roas'] > 3:     score += 2
            elif row['roas'] > 1:     score += 1

            # Orders weighting (0-3 points)
            if row['orders'] > 20:    score += 3
            elif row['orders'] > 10:  score += 2
            elif row['orders'] > 0:   score += 1

            return min(score, 10)  # cap at 10

        kw_agg['score'] = kw_agg.apply(compute_score, axis=1)

        # --- Convert to list of dicts for JSON ---
        summary['keyword_performance'] = kw_agg.to_dict(orient='records')
        summary['keywords'] = kw_agg['customer_search_term'].tolist()

        conn.close()
        return summary

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
summary = get_campaign_summary_with_keyword_score('ads_data.db',"UTELT4BT2754-600GMBLACK - Group C")
print(json.dumps(summary, indent=4, sort_keys=True))

{
    "campaign_info": {
        "bidding_strategy": "Dynamic bids - down only",
        "budget_amount": "$30.0",
        "campaign_name": "UTELT4BT2754-600GMBLACK - Group C",
        "country": "United States",
        "currency": "USD",
        "portfolio_name": "Towel",
        "program_type": "Sponsored Products",
        "retailer": "Amazon",
        "status": "ENABLED",
        "targeting_type": "Manual targeting"
    },
    "campaign_performance": {
        "7_day_total_orders_(#)": 76,
        "7_day_total_sales": "$2964.02",
        "campaign_name": "UTELT4BT2754-600GMBLACK - Group C",
        "click-thru_rate_(ctr)": "1.121%",
        "clicks": 294,
        "cost_per_click_(cpc)": "$1.5181973",
        "end_date": "Jan 04, 2026",
        "impressions": 26236,
        "last_year_clicks": null,
        "last_year_cost_per_click_(cpc)": "$0.0",
        "last_year_impressions": null,
        "last_year_spend": null,
        "spend": "$446.35",
        "start_date": "Dec 06, 2025

In [26]:
def add_csv_to_sqlite(db_path, csv_path):
    """
    Reads a new CSV of keyword-level ad performance and inserts it into SQLite.
    Renames overlapping columns to avoid conflicts.
    """
    # --- Load CSV ---
    df = pd.read_csv(csv_path)

    # --- Define columns already in DB to rename ---
    existing_columns = [
        'campaign_name', 'ad_group_name', 'impressions', 'clicks', 'spend', 'sales', 'orders', 'units',
        'ctr', 'acos', 'cpc', 'roas', 'conversion_rate'
    ]

    # Map CSV columns to DB-friendly names
    rename_map = {}
    for col in df.columns:
        col_lower = col.lower().replace(' ', '_').replace('(informational_only)', '').replace('(', '').replace(')', '')
        # rename if conflicts
        if col_lower in existing_columns:
            rename_map[col] = f'ad_{col_lower}'
        else:
            rename_map[col] = col_lower

    df.rename(columns=rename_map, inplace=True)

    # --- Connect to SQLite ---
    conn = sqlite3.connect(db_path)

    # --- Write to DB ---
    df.to_sql('keyword_ad_performance', conn, if_exists='replace', index=False)

    conn.close()
    print("CSV added to SQLite table 'keyword_ad_performance' successfully!")

# -------------------------
# Example usage:
db_path = "ads_data.db"
csv_path = "bid_data.csv"
add_csv_to_sqlite(db_path, csv_path)

CSV added to SQLite table 'keyword_ad_performance' successfully!


In [28]:
def fill_null_bid_with_default(db_path):
    """
    Updates the 'keyword_ad_performance' table in SQLite:
    replaces NULL 'bid' values with 'ad_group_default_bid_' from the same row.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        cursor.execute("""
            UPDATE keyword_ad_performance
            SET bid = [ad_group_default_bid_]
            WHERE bid IS NULL;
        """)
        conn.commit()
        print(f"Updated {cursor.rowcount} rows with default bid.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        conn.close()

# -------------------------
# Example usage:
db_path = "ads_data.db"
fill_null_bid_with_default(db_path)

Updated 36599 rows with default bid.


In [33]:
def get_campaign_summary(db_path, campaign_name):
    """
    Returns a campaign summary:
    - campaign performance
    - campaign info
    - aggregated keyword performance
    - bid from keyword_ad_performance
    - score out of 10
    """
    summary = {}

    try:
        conn = sqlite3.connect(db_path)

        # --- Campaign performance ---
        campaign_perf = pd.read_sql_query(
            "SELECT * FROM CampaignPerformance WHERE campaign_name = ?",
            conn,
            params=(campaign_name,)
        )
        summary['campaign_performance'] = campaign_perf.to_dict(orient='records')[0] if not campaign_perf.empty else None

        # --- Campaign info ---
        campaign_info = pd.read_sql_query(
            "SELECT * FROM Campaign WHERE campaign_name = ?",
            conn,
            params=(campaign_name,)
        )
        summary['campaign_info'] = campaign_info.to_dict(orient='records')[0] if not campaign_info.empty else None

        # --- Keyword performance ---
        kw_perf = pd.read_sql_query(
            "SELECT * FROM KeywordPerformance WHERE campaign_name = ?",
            conn,
            params=(campaign_name,)
        )
        if kw_perf.empty:
            summary['keyword_performance'] = []
            summary['keywords'] = []
            conn.close()
            return summary

        # --- Aggregate metrics by ad_group_name + customer_search_term ---
        kw_agg = kw_perf.groupby(['ad_group_name', 'customer_search_term']).agg(
            impressions=pd.NamedAgg(column='impressions', aggfunc='sum'),
            clicks=pd.NamedAgg(column='clicks', aggfunc='sum'),
            spend=pd.NamedAgg(column='spend', aggfunc='sum'),
            sales=pd.NamedAgg(column='7_day_total_sales', aggfunc='sum'),
            orders=pd.NamedAgg(column='7_day_total_orders_#', aggfunc='sum')
        ).reset_index()

        # --- Compute CTR, CPC, ROAS ---
        kw_agg['ctr'] = kw_agg['clicks'] / kw_agg['impressions'].replace(0, 1)
        kw_agg['cpc'] = kw_agg['spend'] / kw_agg['clicks'].replace(0, 1)
        kw_agg['roas'] = kw_agg['sales'] / kw_agg['spend'].replace(0, 1)

        # --- Merge bid from keyword_ad_performance ---
        ad_perf = pd.read_sql_query(
            """
            SELECT campaign_name_ AS campaign_name,
                   ad_group_name_ AS ad_group_name,
                   keyword_text AS customer_search_term,
                   bid,
                   ad_group_default_bid_ AS default_bid
            FROM keyword_ad_performance
            WHERE campaign_name_ = ?
            """,
            conn,
            params=(campaign_name,)
        )

        # Fill missing bid with default bid
        ad_perf['bid'] = ad_perf['bid'].fillna(ad_perf['default_bid'])

        # Merge bids into aggregated keyword metrics
        kw_agg = kw_agg.merge(
            ad_perf[['ad_group_name', 'customer_search_term', 'bid']],
            on=['ad_group_name', 'customer_search_term'],
            how='left'
        )

        # --- Compute score out of 10 ---
        def compute_score(row):
            score = 0
            # CTR (0-4 points)
            if row['ctr'] > 0.1:      score += 4
            elif row['ctr'] > 0.05:   score += 3
            elif row['ctr'] > 0.02:   score += 2
            elif row['ctr'] > 0:      score += 1

            # ROAS (0-3 points)
            if row['roas'] > 5:       score += 3
            elif row['roas'] > 3:     score += 2
            elif row['roas'] > 1:     score += 1

            # Orders (0-3 points)
            if row['orders'] > 20:    score += 3
            elif row['orders'] > 10:  score += 2
            elif row['orders'] > 0:   score += 1

            return min(score, 10)

        kw_agg['score'] = kw_agg.apply(compute_score, axis=1)

        # --- Convert to JSON-friendly format ---
        summary['keyword_performance'] = kw_agg.to_dict(orient='records')
        summary['keywords'] = kw_agg['customer_search_term'].tolist()

        conn.close()
        return summary

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
        
summary = get_campaign_summary('ads_data.db',"UTELT4BT2754-600GMBLACK - Group C")
print(json.dumps(summary, indent=4, sort_keys=True))

{
    "campaign_info": {
        "bidding_strategy": "Dynamic bids - down only",
        "budget_amount": "$30.0",
        "campaign_name": "UTELT4BT2754-600GMBLACK - Group C",
        "country": "United States",
        "currency": "USD",
        "portfolio_name": "Towel",
        "program_type": "Sponsored Products",
        "retailer": "Amazon",
        "status": "ENABLED",
        "targeting_type": "Manual targeting"
    },
    "campaign_performance": {
        "7_day_total_orders_(#)": 76,
        "7_day_total_sales": "$2964.02",
        "average_time_in_budget": "95.88%",
        "campaign_name": "UTELT4BT2754-600GMBLACK - Group C",
        "click-thru_rate_(ctr)": "1.121%",
        "clicks": 294,
        "cost_per_click_(cpc)": "$1.5181973",
        "end_date": "Jan 04, 2026",
        "impressions": 26236,
        "last_year_clicks": null,
        "last_year_cost_per_click_(cpc)": "$0.0",
        "last_year_impressions": null,
        "last_year_spend": null,
        "spend": "$